### This notebook file is used to make prediction for 2020 Olympic via CNN model

In [1]:
# Import required libraries and read test and train data set
import pandas as pd
import numpy as np

## Summer Olympics 2020

In [2]:
summer_dataset_2020 = pd.read_csv("Generated Dataset/2020 Olympics.csv")
# log the GDP and Population
summer_dataset_2020["PopulationSize"] = np.log(summer_dataset_2020["PopulationSize"])
summer_dataset_2020["GDPPerCapita"] = np.log(summer_dataset_2020["GDPPerCapita"])
summer_dataset_2020.head(10)

,CountryCode,Year,OlympicsSeason,TotalParticipants,GoldMedals,SilverMedals,BronzeMedals,TotalMedals,Win%,GDPPerCapita,...,PrevTotalParticipants,PrevGoldMedals,PrevSilverMedals,PrevBronzeMedals,PrevTotalMedals,PrevWinterTotalParticipants,PrevWinterGoldMedals,PrevWinterSilverMedals,PrevWinterBronzeMedals,PrevWinterTotalMedals
0,AFG,2016,NaN,3,NaN,NaN,NaN,NaN,NaN,6.091294,...,3,0,0,0,0,0,0,0,0,0
1,ALB,2016,NaN,6,NaN,NaN,NaN,NaN,NaN,8.350814,...,6,0,0,0,0,2,0,0,0,0
2,ALG,2016,NaN,74,NaN,NaN,NaN,NaN,NaN,8.462153,...,74,0,2,0,2,0,0,0,0,0
3,AND,2016,NaN,4,NaN,NaN,NaN,NaN,NaN,10.754391,...,4,0,0,0,0,12,0,0,0,0
4,ANG,2016,NaN,26,NaN,NaN,NaN,NaN,NaN,8.380535,...,26,0,0,0,0,0,0,0,0,0
5,ANT,2016,NaN,9,NaN,NaN,NaN,NaN,NaN,9.720109,...,9,0,0,0,0,0,0,0,0,0
6,ARG,2016,NaN,232,NaN,NaN,NaN,NaN,NaN,9.059287,...,232,21,1,0,22,20,0,0,0,0
7,ARM,2016,NaN,34,NaN,NaN,NaN,NaN,NaN,8.521416,...,34,1,3,0,4,6,0,0,0,0
8,ARU,2016,NaN,7,NaN,NaN,NaN,NaN,NaN,10.326632,...,7,0,0,0,0,0,0,0,0,0
9,ASA,2016,NaN,4,NaN,NaN,NaN,NaN,NaN,9.581031,...,4,0,0,0,0,0,0,0,0,0


In [3]:
summer_dataset = pd.read_csv("Generated Dataset/summer_dataset.csv")
summer_dataset = summer_dataset.drop(["Unnamed: 0"], axis = 1)
# if the data is too few (less than 3), simply ignore that country
NOC_values = dict(summer_dataset.groupby("CountryCode")["CountryCode"].value_counts())
ignore_value = []
for key in NOC_values:
    if NOC_values[key] <= 3:
        ignore_value.append(key[0])

In [4]:
summer_dataset_2020 = pd.DataFrame(summer_dataset_2020[(~(summer_dataset_2020["CountryCode"].isin(ignore_value)))]).reset_index(drop = True)

In [5]:
predictors = ["HostCity","GDPPerCapita","PopulationSize","TotalParticipants", "PrevTotalParticipants", "PrevGoldMedals", "PrevSilverMedals", "PrevBronzeMedals", "PrevTotalMedals",\
                      "PrevWinterTotalParticipants", "PrevWinterGoldMedals", "PrevWinterSilverMedals", "PrevWinterBronzeMedals", "PrevWinterTotalMedals"]

In [6]:
y_test = pd.DataFrame(summer_dataset_2020["TotalMedals"])
X_test = pd.DataFrame(summer_dataset_2020[predictors])

In [7]:
from tensorflow import keras

In [8]:
#to load a model
NAME = "{}-layers".format(6)
model = keras.models.load_model('Keras Models/{}.model'.format(NAME))

In [9]:
y_test_pred = model.predict(np.array(X_test))
y_test_pred = list(y_test_pred)
for i in range(len(y_test_pred)):
    if y_test_pred[i] < 0:
        y_test_pred[i] = 0
y_test_pred = np.round(y_test_pred).astype("int")

In [10]:
NOC_neat = list(summer_dataset_2020["CountryCode"].unique())
NOC_neat.sort()

In [11]:
y_test_pred_df = pd.DataFrame(y_test_pred)
noc_df = pd.DataFrame(NOC_neat)
result_summer = pd.concat([noc_df, y_test_pred_df], axis = 1)
result_summer.columns = ["Country", "Predicted Medals"]

In [12]:
print(result_summer.shape)
result_summer.head()

(194, 2)


,Country,Predicted Medals
0,AFG,4
1,ALB,4
2,ALG,7
3,AND,4
4,ANG,4


In [13]:
# create a csv file
result_summer.to_csv("Results/Summer 2020 - Keras CNN Model.csv", index = 0)